In [1]:
%%html
<style>.text_cell .rendered_html * {direction: rtl; text-align: right;}</style>

In [2]:
import time

در این پروژه قصد داریم تا با استفاده از الگوریتم های جستجوی آگاهانه و ناآگاهانه مسیله مربوطه را حل کنیم و در یک ماز در ضمن رسیدن به نقطه خروج، توپ ها را نیز جابجا کنیم

قطعه کد زیر کلاس توپ ها را نشان میدهد که برای نگه داشتن ساختار تود و محل قرار گیری آنها و هدف آنها استفاده میشود.

In [3]:
class Ball(object):
    def __init__(self, _startX, _startY, _endX, _endY):
        self.startX = _startX
        self.startY = _startY
        self.endX = _endX
        self.endY = _endY
    
    def __eq__(self, other):
        if(self.startX == other.startX and self.startY == other.startY 
           and self.endX == other.endX and self.endY == other.endY):
            return True
        else:
            return False

قطعه کد زیر کلاس استیت ها را نشان میدهد که برای نگه داشتن هر استیت از مسئله بکار میرود.
برای حل این مسئله چه در روش های جستجوی آگاهانه و ناآگاهانه از این کلاس استفاده میکنیم و مدل سازی مسئله را بدین صورت انجام میدهیم که:

initial state: استیتی که در نقطه ی شروع هستیم و تعداد حرکاتمان صفر بوده و هیچ توپی در کیف نداریم و هیچ توپی نیز در جای درست خود قرار نگرفته است

goal state: استیتی که در آن در نقطه ی پایان قرار داریم و تعدادی حرکت انجام داده ایم و همه ی توپ ها در جای درست قرارگرفته اند و هیچ توپی در کیف نداریم

actions: حرکت هایی که در یک استیت میتوانیم انجام دهیم تا به استیت بعد برویم به ترتیب برداشتن توپ(اگر در آن محل باشد) ، راست رفتن، چپ رفتن، بالا و پایین رفتن است.که متد getActions این حرکات را انجام داده و لیستی از استیت های 
جدید که از استیت حاضر به آنها میرسیم را برمیگرداند.
درconstrcutor این کلاس اگر در محلی باشیم باشیم که جای مناسب برای یکی از گلوله هایی است که در کیف داریم آن را خالی میکنیم.

همچنین برای راحت تر چک کردن تساوی دو توپ یا دو دو استیت برای هردو کلاس اپراتور = اورلود شده است.


In [4]:
class State(object):
    def __init__(self, _x, _y, allBalls, _ballsInBag, _correctBalls, _move):
        self.x = _x
        self.y = _y
        self.ballsInBag = _ballsInBag
        self.correctBalls = _correctBalls
        self.move = _move
        
        for ball in self.ballsInBag:
            if(self.x == allBalls[allBalls.index(ball)].endX and self.y == allBalls[allBalls.index(ball)].endY):
                self.ballsInBag.remove(ball)
                self.correctBalls.append(ball)

    def __eq__(self, other):
        if(self.x != other.x or self.y != other.y or
           len(self.ballsInBag) != len(other.ballsInBag) or len(self.correctBalls) != len(other.correctBalls)):
            return False
        
        if(len(self.ballsInBag) == 0 and len(self.correctBalls) == 0):
            return True
        
        for ball in self.ballsInBag:
            if(not(ball in other.ballsInBag)):
                return False
        
        for ball in self.correctBalls:
            if(not(ball in other.correctBalls)):
                return False
        return True
        
    def getActions(self, gameMap, balls, capacity, dim):
        actions= []
        
        #get ball
        if(len(self.ballsInBag) < capacity):
            for ball in balls:
                if(self.x == ball.startX and self.y == ball.startY and 
                    not(ball in self.correctBalls) and not(ball in self.ballsInBag)):
                    actions.append(State(self.x, self.y, balls, self.ballsInBag.copy() + [ball], 
                                         self.correctBalls.copy(), self.move + 1))
        #right
        if(self.y + 1 < dim[1] and gameMap[self.x][self.y + 1] != '*'):
            actions.append(State(self.x, self.y + 1, 
                            balls, self.ballsInBag.copy(), self.correctBalls.copy(), self.move + 1))
        #left
        if(self.y - 1 >= 0 and gameMap[self.x][self.y - 1] != '*'):
            actions.append(State(self.x, self.y - 1, 
                            balls, self.ballsInBag.copy(), self.correctBalls.copy(), self.move + 1))
        #up
        if(self.x - 1 >= 0 and gameMap[self.x - 1][self.y] != '*'):
            actions.append(State(self.x - 1, self.y, 
                            balls, self.ballsInBag.copy(), self.correctBalls.copy(), self.move + 1))
        #down
        if(self.x + 1 < dim[0] and gameMap[self.x + 1][self.y] != '*'):
            actions.append(State(self.x + 1, self.y, 
                            balls, self.ballsInBag.copy(), self.correctBalls.copy(), self.move + 1))


        return actions
    
    def printState(self):
        print("x: ", self.x, "y: ", self.y, "bagSize: ", len(self.ballsInBag))

In [5]:
def printSolution(move, numOfStates, numOfDiffStates, startTime, endTime):
    print("move: %d" % move)
    print("number of states: %d" % numOfStates)
    print("number of different states: %d" % numOfDiffStates)
    print("time: %f s" % ((endTime - startTime)))

قطعه کد زیر جستجوی ناآگاهانه bfs است که در آن همه ی استیت ها باهم، هم عمق جلو میروند .

این روش نسبت به روش ids و یکی از heuristic ها حافظه بیشتری مصرف میکند اما نسبت به ids 
سریع تر است و زمان اجرای آن از A*بیشتر است

در این روش بطور مداوم چک میکینم که استیت هایی را که قبلا دیده ایم دیگر بررسی نکنیم.

برای پیدا کردن تعداد استیا های مجزای دیده شده از آنجا که هیچ گاه استیت تکراری دیده نمیشود هر بار که حلقه اجرا میشود یکی به تعداد استیت های دیده شده ی مجزا اضافه میکینیم اما برای پیدا کردن کل تعداد استیت های دیده شده هر بار تعداد استیت های جدید را که توسط یک استیت اضافه میشوند را میشماریم.

در کل این پروژه تعداد حرکت ها در خود استیت ها نگه داری میشود و هنگامی که یم استیت جدید در تابع gatActions که در بالانر توضیح داده شده است ساخته میشود یک حرکت نسبت به استیت قبل به آن اضافه میشود.

In [6]:
def bfs(initialState, finaleState, gameMap, balls, capacity, dim):
    numOfStates = 0
    numOfDiffStates = 0
    startTime = time.time()
    

    frontier = [initialState]
    explored = []

    while(True):
        if frontier:
            currentState = frontier.pop(0)
        else:
            return
        numOfDiffStates += 1
        explored.append(currentState)
        actions = currentState.getActions(gameMap, balls, capacity, dim)
        numOfStates += len(actions)
        for action in actions:
            if(not(action in frontier) and not(action in explored)):
                if(action == finaleState):
                    printSolution(action.move, numOfStates, numOfDiffStates, startTime, time.time())
                    return
                else:
                    frontier.append(action)

        

قطعه کد زیر جستجوی ناآگاهانه ids است که در آن همه ی استیت ها باهم، هم عمق جلو نمیروند و ابتدا یک مسیر تا انتها رفته و سپس باقی مسیر ها بررسی میشوند و اگر در حین این کار از عمق محدود شده فراتر بخواهد برود به آن اجازه داده نمیشود .

این روش نسبت به روش bfs  حافظه کمتری مصرف میکند اما زمان اجرای آن از باقی روش ها بیشتر است

در این روش بطور مداوم چک میکینم که استیت هایی را که قبلا دیده ایم دیگر بررسی نکنیم.

برای پیدا کردن تعداد استیت های مجزای دیده شده از آنجا که هیچ گاه استیت تکراری دیده نمیشود هر بار که حلقه اجرا میشود یکی به تعداد استیت های دیده شده ی مجزا اضافه میکینیم اما برای پیدا کردن کل تعداد استیت های دیده شده هر بار تعداد استیت های جدید را که توسط یک استیت اضافه میشوند را میشماریم.

در کل این پروژه تعداد حرکت ها در خود استیت ها نگه داری میشود و هنگامی که یم استیت جدید در تابع gatActions که در بالانر توضیح داده شده است ساخته میشود یک حرکت نسبت به استیت قبل به آن اضافه میشود.

پیاده سازی روش dfsبه صورت بازگشتی است اما برای صرف زمان کم تر از لیست در زبان پایتون به صورت یک stack استفاده کرده ایم تا همان کارایی که باید بیشترین عمق(یعنی آخرین اسنیتی که آمده و روی stack( است بسط داده شود.

همچنین برای سریع تر شدن کار در هر باری که قرار است dfs تا یک عمق مشخص انجام شود استیت هایی که دقبقا در یک عمق بالانر قرار دارند در لیست boundary نگه داری میوشند تا در دور بعدی عمق  های پایین تر دوباره بررسی نشوند و از آن عمق به بعد به بررسی ادامه داده شود.

In [7]:
# def dfs(currentState, finaleState, gameMap, balls, capacity, dim, maxDepth, startTime,
#         numOfStates, numOfDiffStates, visited):
    
# #     visited.append(currentState)
#     if (maxDepth <= 0):
#         return False
    
#     if(currentState == finaleState):
#         printSolution(currentState.move, numOfStates, numOfDiffStates, startTime, time.time())
#         return True
    
#     actions = currentState.getActions(gameMap, balls, capacity, dim)
#     for action in actions:
# #         if(not(action in visited) and action.move < maxDepth):
#         if(dfs(action, finaleState, gameMap, balls, capacity, dim, maxDepth-1, startTime,
#         numOfStates, numOfDiffStates, visited)):
#             return True
#     return False

# def ids(initialState, finaleState, gameMap, balls, capacity, dim, maxDepth):
#     numOfStates = 0
#     numOfDiffStates = 0
#     startTime = time.time()
    
#     for i in range(maxDepth):
#         if(dfs(initialState, finaleState, gameMap, balls, capacity, dim, maxDepth, startTime,
#         numOfStates, numOfDiffStates, [])):
#             return
        
        
    
def ids(initialState, finaleState, gameMap, balls, capacity, dim, maxDepth):
    numOfStates = 0
    numOfDiffStates = 0
    startTime = time.time()
    
    boundary = [initialState]
    visited = [initialState]
    
    for i in range(maxDepth):
        stack = boundary.copy()
        boundary = []
        
        while(len(stack)):
            currentState = stack[-1]
            stack.pop()
            numOfDiffStates += 1
            
            if(currentState == finaleState):
                printSolution(currentState.move, numOfStates, numOfDiffStates, startTime, time.time())
                return

            actions = currentState.getActions(gameMap, balls, capacity, dim)
            numOfStates += len(actions)
            for action in actions:
                if(not(action in visited)):
                    visited.append(action)
                    if(currentState.move <= i):
                        stack.append(action)
                    else:
                        boundary.append(action)
                    

قطعه کد زیر پیادی سازی روش  A* با یکی از تابع های heurisitc است .

این روش نسبت به روش های قبلی سریع تر بوده و حافظه ی کمتری مصرف میکند.

در این جا تابع heuristic ای که استفاده میکنیم بردین صورت است که از بین استیت هایی که در frontier قرار دارند استیتی انتخاب میشود که مجموع تعداد حرکات آن با قاصله آن استیت بصورت مستقیم تا مقصد و مجموع فاصله ی مستقیم توپ هایی که داخل کیف در این استیت نیستنتد مینیمم باشد.

این تابع heuristic مسلما هم admissible و هم consistent میباشد زیرا هزینه ی اصلی ای که باید طی شود از فاصله های مستقیم بزگتر یا مساوی است و همچنین این تابع افزایشی است یعنی برای استیت های که فاصله بیشتری از استیت نهایی دارند(چه از نظر فاصله چه از نظر توپ ها و فواصل آنها) مقدار این تابع نیز بیشتر است.

In [8]:
def findCost(state, balls, finaleState):
    cost = state.move
    cost = cost + abs(state.x - finaleState.x) + abs(finaleState.y - state.y)
    for ball in balls:
        if(not(ball in state.correctBalls)):
            cost = cost + abs(ball.startX - ball.endX) + abs(ball.startY - ball.endY)
    return cost

def getPriorState(frontier, balls, finaleState):
    minCost = findCost(frontier[0], balls, finaleState)
    priorState = frontier[0]
    for state in frontier:
        cost = findCost(state, balls, finaleState)
        if(cost < minCost):
            priorState = state
            
    frontier.remove(priorState)
    return priorState

def heurstic_1(initialState, finaleState, gameMap, balls, capacity, dim):
    numOfStates = 0
    numOfDiffStates = 0
    startTime = time.time()
    
    frontier = [initialState]
#     explored = [initialState]
    explored = []
    while(True):
        currentState = getPriorState(frontier, balls, finaleState)
        numOfDiffStates += 1
        
        explored.append(currentState)
        
        actions = currentState.getActions(gameMap, balls, capacity, dim)
        numOfStates += len(actions)
        for action in actions:
            if(not(action in frontier) and not(action in explored)):
                if(action == finaleState):
                    printSolution(action.move, numOfStates, numOfDiffStates, startTime, time.time())
                    return
                else:
                    frontier.append(action)
        

قطعه کد زیر مربوط به پیادی سازی الگوریتم A* با یک تابع heuristic  دیگر است که در آن رویکرد این تابع به این صورت است که مسئله را از استیتی که در آن قرار دارد با فرض اینکه محدودیتی در تعداد توپ های داخل کیف نداشته باشیم جل میکند و استیتی از مجموعه frontier انتخاب میشود که تعداد حرکات حاصل این مسئله ساده شده کم تر باشد.

این تابع نیز هم admissible و هم consistent است زیرا اولا با اضافه شدن محدودیت تعداد گوی ها در کیف تعداد حرکاتی که باید انجام شود بیشتر میشود یا مساوی میماند و در مرحله بعد این تابع به صورت افزایشی است یعنی اگر استیتی فاصله بیشتری تا استیت نهایی داشته باشد، مقدار هزینه حساب شده نیز بیشتر میشود.

In [9]:
def findCost_2(initialState, finaleState, gameMap, balls, capacity, dim):
    numOfStates = 0
    numOfDiffStates = 0
    

    frontier = [initialState]
    explored = []

    while(True):
        if frontier:
            currentState = frontier.pop(0)
        else:
            return numOfStates
        numOfDiffStates += 1
        explored.append(currentState)
        actions = currentState.getActions(gameMap, balls, capacity, dim)
        numOfStates += len(actions)
        for action in actions:
            if(not(action in frontier) and not(action in explored)):
                if(action == finaleState):
                    return action.move
#                     return numOfStates
                else:
                    frontier.append(action)
                    
def getPriorState_2(frontier, balls, finaleState, gameMap, ballsCount, dim):
    minCost = findCost_2(frontier[0], finaleState, gameMap, balls, ballsCount, dim) + frontier[0].move
    priorState = frontier[0]
    for state in frontier:
        cost = findCost_2(state, finaleState, gameMap, balls, ballsCount, dim) + state.move
        if(cost < minCost):
            priorState = state
            
    frontier.remove(priorState)
    return priorState

def heurstic_2(initialState, finaleState, gameMap, balls, ballsCount, dim):
    numOfStates = 0
    numOfDiffStates = 0
    startTime = time.time()
    
    frontier = [initialState]
    explored = []
    while(True):
        currentState = getPriorState_2(frontier, balls, finaleState, gameMap, ballsCount, dim)
        numOfDiffStates += 1
        
        explored.append(currentState)
        
        actions = currentState.getActions(gameMap, balls, capacity, dim)
        numOfStates += len(actions)
        for action in actions:
            if(not(action in frontier) and not(action in explored)):
                if(action == finaleState):
                    printSolution(action.move, numOfStates, numOfDiffStates, startTime, time.time())
                    return
                else:
                    frontier.append(action)

در این جا تابع heuristic اول را با دو الفای مختلف پیاده سازی کرده ایم که به صورت آزمون و خطا بدست آمده اند.

In [212]:
def findCost_3(state, balls, finaleState):
    g = abs(state.x - finaleState.x) + abs(finaleState.y - state.y) + state.move
    h = 0
    for ball in balls:
        if(not(ball in state.correctBalls)):
            h = h + abs(ball.startX - ball.endX) + abs(ball.startY - ball.endY)
    return g + 500 * h
#     return g + 2 * h

def getPriorState_3(frontier, balls, finaleState):
    minCost = findCost_3(frontier[0], balls, finaleState)
    priorState = frontier[0]
    for state in frontier:
        cost = findCost_3(state, balls, finaleState)
        if(cost < minCost):
            priorState = state
            
    frontier.remove(priorState)
    return priorState

def weightedA(initialState, finaleState, gameMap, balls, capacity, dim):
    numOfStates = 0
    numOfDiffStates = 0
    startTime = time.time()
    
    frontier = [initialState]
#     explored = [initialState]
    explored = []
    while(True):
        currentState = getPriorState_3(frontier, balls, finaleState)
        numOfDiffStates += 1
        
        explored.append(currentState)
        
        actions = currentState.getActions(gameMap, balls, capacity, dim)
        numOfStates += len(actions)
        for action in actions:
            if(not(action in frontier) and not(action in explored)):
                if(action == finaleState):
                    printSolution(action.move, numOfStates, numOfDiffStates, startTime, time.time())
                    return
                else:
                    frontier.append(action)
                    

خواندن ازفایل: در اینحا برای هرکدام از تیست ها با توجه به فرمت ورودی گفته شده از فایل داده ها خوانده میشود و در متغیر های و ساختار داده های مناسب که بالاتر و در ابتدای پروژه ذکر شده است ریخته میشود.

بنابراین در پایان این قطعه کد ما نقشه ی بازی و لیستی از توپ ها را داریم.

In [218]:
#test 1
testFile_1 = open("Tests/1.txt")
content = testFile_1.readlines()

#test 2
# testFile_2 = open("Tests/2.txt")
# content = testFile_2.readlines()

#test 3
# testFile_3 = open("Tests/3.txt")
# content = testFile_3.readlines()

#test 4
# testFile_4 = open("Tests/4.txt")
# content = testFile_4.readlines()

dim = list(map(int, content[0].split()))
startPoint = list(map(int, content[1].split()))
endPoint = list(map(int, content[2].split()))
capacity = int(content[3])
numOfBalls = int(content[4])
ballsPos = [[None for i in range(4)] for i in range(numOfBalls)]
balls = []

for i in range(numOfBalls):
    ballsPos[i] = list(map(int, content[5 + i].split()))
    balls.append(Ball(ballsPos[i][0], ballsPos[i][1], ballsPos[i][2], ballsPos[i][3]))
    
    
gameMap = [[None for i in range(dim[1])] for i in range(dim[0])]
for i in range(dim[0]):
    gameMap[i] = content[5 + numOfBalls + i].split()

تعیین مقادیر اولیه استیت ها

In [219]:
initialState = State(startPoint[0], startPoint[1], [], [], [], 0)
# initialState = State(endPoint[0], endPoint[1], balls, [], balls)
finaleState = State(endPoint[0], endPoint[1], balls, [], balls, 0)


تست bfs

In [220]:
bfs(initialState, finaleState, gameMap, balls, capacity, dim)

move: 20
number of states: 571
number of different states: 225
time: 0.020879 s


تست ids

In [221]:
ids(initialState, finaleState, gameMap, balls, capacity, dim, 100)

move: 20
number of states: 610
number of different states: 245
time: 0.021045 s


تست h1

In [222]:
heurstic_1(initialState, finaleState, gameMap, balls, capacity, dim)

move: 20
number of states: 458
number of different states: 175
time: 0.018424 s


تست h2

In [223]:
heurstic_2(initialState, finaleState, gameMap, balls, numOfBalls, dim)

move: 20
number of states: 336
number of different states: 129
time: 13.316331 s


تست weighted A*

In [215]:
weightedA(initialState, finaleState, gameMap, balls, capacity, dim)

move: 92
number of states: 1019
number of different states: 456
time: 0.051341 s


جدول نهایی:


| تعداد حرکات | تعداد استیت دیده شده | تعداد استیت مجزای دیده شده | میانگین زمان اجرا | test | - |
| :- | :- | :- | :- | -: | :-: |
| 20 | 571 | 225 | 0.018 | 1 | bfs |
| 48 | 9017 | 2301 | 1.155 | 2 | bfs |
| - | - | - | - | 3 | bfs |
| 92 | 1019 | 456 | 0.042 | 4 | bfs |
| 20 | 610 | 245 | 0.022 | 1 | ids |
| 48 | 9022 | 2304 | 1.461 | 2 | ids |
| - | - | - | - | 3 | ids |
| 92 | 1019 | 457 | 0.043 | 4 | ids |
| 20 | 485 | 175 | 0.020 | 1 | h1 |
| 48 | 169 | 48 | 0.006 | 2 | h1 |
| - | - | - | - | 3 | h1 |
| 92 | 1019 | 456 | 0.045 | 4 | h1 |
| 20 | 336 | 129 | 13.248 | 1 | h2 |
| 48 | 154 | 49 | 3m | 2 | h2 |
| - | - | - | - | 3 | h2 |
| 92 | 752 | 347 | 1m,18s | 4 | h2 |
| 32 | 124 | 48 | 0.005 | 1 | alpha = 7 |
| 20 | 169 | 48 | 0.004 | 2 | alpha = 7 |
| 106 | 553 | 179 | 0.07 | 3 | alpha = 7 |
| 92 | 1019 | 456 | 0.046 | 4 | alpha = 7 |
| 20 | 383 | 144 | 0.017 | 1 | alpha = 2 |
| 48 | 169 | 48 | 0.004 | 2 | alpha = 2 |
| 76 | 7007 | 2433 | 0.018 | 3 | alpha = 4 |
| 92 | 1019 | 456 | 0.044 | 4 | alpha = 2 |

در نهایت از خطاهای این پروژه میتوان به میزان زمان مصرفی زیاد برای تست ها اشاره کرد که بررسی آنها را سخت میکند.
از جمله خطاهای دیگر استفاده از اورلودینگ اپراتور های استفاده شده است که باید با دقت انجام شود.
